In [9]:
# ================================================
# 1. Imports
# ================================================
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
import numpy as np

In [18]:
# ================================================
# 2. Load datasets (train + eval)
# ================================================
train_df = pd.read_csv("/Users/kalpitpatel/CascadeProjects/housing-price-regressor/data/processed/feature_engineered_train.csv")
eval_df = pd.read_csv("/Users/kalpitpatel/CascadeProjects/housing-price-regressor/data/processed/feature_engineered_eval.csv")

In [20]:
'''
# ================================================
# 3. Drop high VIF features (both train + eval)
# ================================================
high_vif_features = [
    "median_sale_price" #highest correlation to 'price' => data leakage
]
train_df.drop(columns=high_vif_features, inplace=True)
eval_df.drop(columns=high_vif_features, inplace=True)
'''

'\n# ================================================\n# 3. Drop high VIF features (both train + eval)\n# ================================================\nhigh_vif_features = [\n    "median_sale_price" #highest correlation to \'price\' => data leakage\n]\ntrain_df.drop(columns=high_vif_features, inplace=True)\neval_df.drop(columns=high_vif_features, inplace=True)\n'

In [3]:
# ================================================
# 4. Define target & features
# ================================================
target = "price"
X_train = train_df.drop(columns=[target])
y_train = train_df[target]

X_eval = eval_df.drop(columns=[target])
y_eval = eval_df[target]

In [11]:
# ================================================
# 5. Standardization (fit on train, transform eval)
# ================================================
scaler = StandardScaler()
X_train_scaled_old = scaler.fit_transform(X_train)
X_eval_scaled_old  = scaler.transform(X_eval)
print("NaN values before imputation:")
print(np.isnan(X_train_scaled_old).sum(axis=0))
imputer = SimpleImputer(strategy='mean')
X_train_scaled = imputer.fit_transform(X_train_scaled_old)
X_eval_scaled = imputer.transform(X_eval_scaled_old)
print("NaN values after imputation:")
print(np.isnan(X_train_scaled).sum(axis=0))
print(np.isnan(X_eval_scaled).sum(axis=0))

/Users/kalpitpatel/CascadeProjects/housing-price-regressor/venv/lib/python3.14/site-packages/sklearn/utils/extmath.py:1207: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/Users/kalpitpatel/CascadeProjects/housing-price-regressor/venv/lib/python3.14/site-packages/sklearn/utils/extmath.py:1212: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/Users/kalpitpatel/CascadeProjects/housing-price-regressor/venv/lib/python3.14/site-packages/sklearn/utils/extmath.py:1236: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


NaN values before imputation:
[     0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0 576815 576815      0      0]
NaN values after imputation:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


/Users/kalpitpatel/CascadeProjects/housing-price-regressor/venv/lib/python3.14/site-packages/sklearn/impute/_base.py:641: UserWarning: Skipping features without any observed values: [35 36]. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/Users/kalpitpatel/CascadeProjects/housing-price-regressor/venv/lib/python3.14/site-packages/sklearn/impute/_base.py:641: UserWarning: Skipping features without any observed values: [35 36]. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


In [13]:
# ================================================
# 6. Train & Evaluate Models
# ================================================

# --- Linear Regression ---
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)
y_pred_lr = lr.predict(X_eval_scaled)

print("Linear Regression:")
print(" MAE:", mean_absolute_error(y_eval, y_pred_lr))
print(" RMSE:", np.sqrt(mean_squared_error(y_eval, y_pred_lr)))
print(" R²:", r2_score(y_eval, y_pred_lr))

Linear Regression:
 MAE: 53811.93813400999
 RMSE: 121336.1346929606
 R²: 0.8862267031700102


In [14]:
# --- Ridge Regression ---
ridge = Ridge(alpha=1.0)
ridge.fit(X_train_scaled, y_train)
y_pred_ridge = ridge.predict(X_eval_scaled)

print("\nRidge Regression:")
print(" MAE:", mean_absolute_error(y_eval, y_pred_ridge))
print(" RMSE:", np.sqrt(mean_squared_error(y_eval, y_pred_ridge)))
print(" R²:", r2_score(y_eval, y_pred_ridge))


Ridge Regression:
 MAE: 53811.114660445484
 RMSE: 121338.02551498353
 R²: 0.886223157206856


In [15]:
# --- Lasso Regression ---
lasso = Lasso(alpha=0.1)
lasso.fit(X_train_scaled, y_train)
y_pred_lasso = lasso.predict(X_eval_scaled)

print("\nLasso Regression:")
print(" MAE:", mean_absolute_error(y_eval, y_pred_lasso))
print(" RMSE:", np.sqrt(mean_squared_error(y_eval, y_pred_lasso)))
print(" R²:", r2_score(y_eval, y_pred_lasso))


Lasso Regression:
 MAE: 54117.426071230286
 RMSE: 121604.47823437209
 R²: 0.8857229111303108


/Users/kalpitpatel/CascadeProjects/housing-price-regressor/venv/lib/python3.14/site-packages/sklearn/linear_model/_coordinate_descent.py:716: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.114e+15, tolerance: 5.209e+12
  model = cd_fast.enet_coordinate_descent(


In [16]:
# --- ElasticNet ---
elastic = ElasticNet(alpha=0.1, l1_ratio=0.5)
elastic.fit(X_train_scaled, y_train)
y_pred_elastic = elastic.predict(X_eval_scaled)

print("\nElasticNet Regression:")
print(" MAE:", mean_absolute_error(y_eval, y_pred_elastic))
print(" RMSE:", np.sqrt(mean_squared_error(y_eval, y_pred_elastic)))
print(" R²:", r2_score(y_eval, y_pred_elastic))


ElasticNet Regression:
 MAE: 54234.249320614595
 RMSE: 122295.84870428537
 R²: 0.8844197946433394
